<a href="https://colab.research.google.com/github/pedrogengo/CISI_BM25/blob/main/notebooks/Developing_BM25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BM25: Entendimento e implementação

Nesse notebook iremos entender como é calculado o score do BM25 e depois iremos implementá-lo.

## 1. Explicação

O BM25 tenta resolver o problema de representar textos em um espaço que seja possível encontrar os documentos mais próximos/semelhantes à uma query. Para isso, assim como o TFIDF, fazemos uso de dois conceitos principais:

- Frequência do termo: quantidade de vezes que um termo/token aparece em um documento;
- Frequência de documentos inversa: avalia em quantos documentos um mesmo termo aparece. Logo, se um termo aparece em muito documentos queremos que ele tenha um peso menor, por isso usamos o inverso dessa frequência.

A principal diferença do BM25 para o TFIDF é que temos duas constantes ($k_1$ e $b$), onde $k_1$ é a constante que determina a velocidade de saturação da frequência do termo e $b$ diz qual o impacto do tamanho de um documento em relação a média de tamanhos.

A equação que descreve o BM25 socre é a seguinte:

\begin{align}
\mbox{BM25}(D, Q) = \sum_{i=1}^n IDF(q_i, D) \frac{f(q_i, D) \cdot (k_1 + 1)}{f(q_i, D) + k_1 \cdot (1-b + b \cdot |D|/d_{avg}))}
\end{align}


Onde:

 - $f(q_i, D)$ é o número de vezes que o termo $q_i$ ocorre no documento $D$.
 - $|D|$ é o número de termos no documento $D$.
 - $d_{avg}$ é quantidade média de palavras em todos os documentos da coleção.
 - $b$ and $k_1$ são hiperparâmetros do BM25.


 Caso $k_1$ seja 0, teremos a mesma equação do TFIDF para cada termo da query. Mas o que acontece conforme aumentamos o valor de $k_1$? Iremos manter o valor de $b$ como zero e usaremos apenas a equação $\frac{f(q_i, D) \cdot (k_1 + 1)}{f(q_i, D) + k_1}$ para nossa análise.

In [15]:
import numpy as np
import plotly.graph_objects as go

def tf(freq, k):
  return (freq * k) / (freq + k)


f = np.arange(10)

fig = go.Figure()
for k in range(1, 15):
  fig.add_trace(go.Scatter(x=f, y=tf(f, k), name=k))
fig.show()

Podemos ver com o gráfico acima que quanto menor o valor de $k_1$, mais rápido ocorre a saturação. Isso quer dizer que se queremos dar importância a quantidade de vezes que um termo aparece no documento, devemos usar um valor mais alto para $k_1$. Já, se entendemos que, por exemplo, mais que 10 aparições já não nos diz muita coisa, usamos um valor de $k_1$ que sature perto dessa região.

Já o parâmetro $b$, como já foi dito, indica qual a importância queremos dar ao tamanho do documento. Um exemplo disso é: Se temos dois documentos onde um mesmo termo aparece 2 vezes em cada um podemos dizer que então o score daqueles documentos para aquele termo deveria ser o mesmo? E se eu trouxer uma nova informação onde o primeiro documento tem 10 palavras e o segundo 200. Será que deveriam ter o mesmo score? Ou o maior, por ter mais palavras, tem mais chance de aparecer mais vezes e deveriamos de alguma forma normalizar essas aparições? O parâmetro $b$ serve para indicar o "peso" que queremos colocar para esse cenário.

Vamos agora fazer uma análise fixando $k_1$ em 1 e variando o valor de $b$ e também do $|D|/d_{avg}$.


In [42]:
from plotly.subplots import make_subplots

def tf(freq, k, b, norm):
  return (freq * k) / (freq + (k * (1 - b + b * norm)))


f = np.arange(10)

fig = make_subplots(rows=10, cols=1, subplot_titles=("Norm=0.5", "Norm=1", "Norm=1.5", "Norm=2", "Norm=2.5", "Norm=3", "Norm=3.5", "Norm=4", "Norm=4.5", "Norm=5"))
plot_idx = 1
for norm in np.arange(0.5, 5.5, 0.5):
  for b in np.arange(0, 1, 0.2):
    fig.add_trace(go.Scatter(x=f, y=tf(f, 1, b, norm), legendgroup = str(plot_idx), name=round(b, 2)), row = plot_idx, col=1)
  plot_idx += 1

fig.update_layout(height=5000, width=1000, legend_tracegroupgap=410)
fig.show()

Pelos gráficos acima, podemos notar que, conforme o tamanho do documento se distancia mais da média (para mais), ao aumentar o $b$ penalizamos mais o score, obtendo um score menor. Já ao ter documentos com tamanhos menores que a média, ao aumentar o $b$ aumentamos o score também.

Logo, é muito importante escolher valores de $k_1$ e $b$ de acordo com a sua coleção de documentos!

## 2. Implementação

Teremos duas etapas na implementação:

- Função para criar o index reverso, onde já teremos deixar calculado as frequências e idfs
- Classe do BM25, que irá calcular o score e retornar os top k elementos

In [44]:
import math
import re

def tokenize(text):
    """Tokenize a document or query."""
    words = re.findall(r'\w+', text.lower())
    return words

def build_index(documents):
    """Build an inverted index from the documents."""
    index = {}
    doc_term_freqs = []

    for i, document in enumerate(documents):
        # Tokenize the document
        terms = tokenize(document)

        # Count the term frequencies
        term_freqs = {}
        for term in terms:
            term_freqs[term] = term_freqs.get(term, 0) + 1

        doc_term_freqs.append(term_freqs)

        # Add the document to the index for each term it contains
        for term in term_freqs:
            if term not in index:
                index[term] = []
            index[term].append((i, term_freqs[term]))

    # Calculate the inverse document frequencies
    N = len(documents)
    idfs = {term: math.log(1 + ((N - len(postings) + 0.5) / (len(postings) + 0.5))) for term, postings in index.items()}

    # Return the inverted index and document term frequencies
    return {"doc_term_freqs": doc_term_freqs, "idfs": idfs}

In [45]:
class BM25():

  def __init__(self, index, k, b, tokenizer):
    self.doc_term_freqs = index["doc_term_freqs"]
    self.idfs = index["idfs"]
    self.k = k
    self.b = b
    self.tokenizer = tokenizer
    self.documents_lengths = [self._count_tokens(doc) for doc in self.doc_term_freqs]
    self.avg_doc_len = sum(self.documents_lengths) / len(self.doc_term_freqs)

  def _count_tokens(self, document):
    """Counts the amount of token in a document"""
    total = 0
    for token_count in document.values():
      total += token_count
    return total

  def search(self, query: str, k: int = 10):
    """Returns the top k documents related to the query"""
    scores = []
    tokenized_query = self.tokenizer(query)
    for i in range(len(self.doc_term_freqs)):
      scores.append((self.score(tokenized_query, i), i))
    scores.sort(reverse=True)
    return scores[:k]

  def score(self, tokenized_query, doc_id: int):
    """Calculates bm25 score for a query and a document"""
    score = 0.
    norm = self.documents_lengths[doc_id] / self.avg_doc_len
    for token in tokenized_query:
      if token not in self.doc_term_freqs[doc_id].keys():
        score += 0.
      else:
        term_freq = self.doc_term_freqs[doc_id][token]
        idf = self.idfs[token]

        numerator = idf * term_freq * (self.k + 1)
        denominator = (term_freq * self.k) / (term_freq + (self.k * (1 - self.b + self.b * norm)))

        score += (numerator / denominator)
    
    return score

In [46]:
test_documents = [
    "esse é o primeiro texto",
    "Nesse texto iremos falar sobre os fundamentos da inteligencia artificial",
    "Machine learning é um subcampo da inteligencia artificial",
    "palavras aleatorias oi, hoje, amanha, circo, casa, teto"
]

query_equal = "esse é o primeiro texto"
query_diff = "Quais são as subareas da inteligencia artificial?"

In [48]:
index = build_index(test_documents)
bm25 = BM25(index, 1.5, 0.8, tokenize)

In [52]:
assert bm25.search(query_equal)[0][1] == 0
assert bm25.search(query_diff)[0][1] == 1 and bm25.search(query_diff)[1][1] == 2